# Run the CICI SAFE vSDX

#### Configure the Envrionment

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-819993'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME='yjyao_jupyter1'
export SSH_PRIVATE_KEY=${HOME}/.ssh/id_rsa

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
# NODE_RESERVATION_ID must be for Skylake nodes
export SDX_RESERVATION_ID='b88365d8-9904-4569-9c3c-9fcaf61d79f3'
export CLIENT_RESERVATION_ID='b88365d8-9904-4569-9c3c-9fcaf61d79f3'
export RIAK_RESERVATION_ID='b88365d8-9904-4569-9c3c-9fcaf61d79f3'

#Set the working directory
export WORKING_DIR=${HOME}/work/SDX

#### Start RIAK Server

In [ ]:
cd $WORKING_DIR

RIAK_NETWORK='sharednet1'
RIAK_NODE_NAME=${USERNAME}-riak
RIAK_STACK_NAME=${USERNAME}-riak

echo Creating RIAK server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template riak.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${SDX_RESERVATION_ID}" \
        --parameter "network_name=${RIAK_NETWORK}" \
        --parameter "riak_node_name=${RIAK_NODE_NAME}" \
        --wait \
        ${RIAK_STACK_NAME}

In [ ]:
echo RIAK server creation complete! 
echo ${RIAK_NODE_NAME} status `openstack server show  --format value -c status ${RIAK_NODE_NAME}`

RIAK_PUBLIC_IP=`openstack server show  --format value -c addresses ${RIAK_NODE_NAME} | cut -d " " -f 2`
echo The RIAK se public IP is $RIAK_PUBLIC_IP

#### Start SDX Server

In [ ]:
cd $WORKING_DIR

SDX_NETWORK='sharednet1'
SDX_NODE_NAME=${USERNAME}-sdx
SDX_STACK_NAME=${USERNAME}-sdx

echo Creating SDX server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template sdx.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${SDX_RESERVATION_ID}" \
        --parameter "network_name=${SDX_NETWORK}" \
        --parameter "sdx_node_name=${SDX_NODE_NAME}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --wait \
        ${SDX_STACK_NAME}

In [ ]:
echo SDX server creation complete! 
echo ${SDX_NODE_NAME} status `openstack server show  --format value -c status ${SDX_NODE_NAME}`

SDX_PUBLIC_IP=`openstack server show  --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
echo The SDX se public IP is $SDX_PUBLIC_IP

In [ ]:
SDX_PUBLIC_IP=`openstack server show --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
echo SDX_PUBLIC_IP is $SDX_PUBLIC_IP

result=255
while [[ "$result" != "0" ]]
do 
    sleep 5
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    exit
	result=$?
done

echo COPY your geni pem to cc@$SDX_PUBLIC_IP:~/geni.pem

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo tail /root/boot.log


#### Start ExoGENI Client 1

In [ ]:
cd $WORKING_DIR
CLIENT_NETWORK='sharednet1'
CLIENT_NODE_NAME1=${USERNAME}-client1
CLIENT_STACK_NAME1=${USERNAME}-clien1
GIT_BRANCH='tridentcom'
SLICE_NAME1='c0-tri'
IP_PREFIX1='192.168.10.1/24'
CLIENT_SITE1='TAMU'
SAFE_KEYPAIR1='key_p10'

echo Creating SDX client. This will take several minutes!
openstack stack create  --max-width 80 \
        --template client.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${CLIENT_RESERVATION_ID}" \
        --parameter "network_name=${CLIENT_NETWORK}" \
        --parameter "client_name=${CLIENT_NODE_NAME1}" \
        --parameter "github_branch=${GIT_BRANCH}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --parameter "slice_name=${SLICE_NAME1}"\
        --parameter "ip_prefix=${IP_PREFIX1}"\
        --parameter "client_site=${CLIENT_SITE1}"\
        --parameter "safe_keypair=${SAFE_KEYPAIR1}"\
        --parameter "sdx_server=${SDX_PUBLIC_IP}:8888"\
        --wait \
        ${CLIENT_STACK_NAME1}

In [ ]:
echo SDX client creation complete! 
echo ${CLIENT_NODE_NAME1} status `openstack server show  --format value -c status ${CLIENT_NODE_NAME1}`

CLIENT_PUBLIC_IP1=`openstack server show  --format value -c addresses ${CLIENT_NODE_NAME1} | cut -d " " -f 2`
echo The client public IP is $CLIENT_PUBLIC_IP1

result=255
while [[ "$result" != "0" ]]
do 
    sleep 5
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    exit
	result=$?
done

echo COPY your geni pem to cc@$CLIENT_PUBLIC_IP1:~/geni.pem

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    sudo tail ~/boot.log


#### Start ExoGENI Client 2

In [ ]:
cd $WORKING_DIR
CLIENT_NETWORK='sharednet1'
CLIENT_NODE_NAME2=${USERNAME}-client2
CLIENT_STACK_NAME2=${USERNAME}-clien2
GIT_BRANCH='tridentcom'
SLICE_NAME2='c1-tri'
IP_PREFIX2='192.168.20.1/24'
CLIENT_SITE2='UFL'
SAFE_KEYPAIR2='key_p11'

echo Creating SDX client. This will take several minutes!
openstack stack create  --max-width 80 \
        --template client.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${CLIENT_RESERVATION_ID}" \
        --parameter "network_name=${CLIENT_NETWORK}" \
        --parameter "client_name=${CLIENT_NODE_NAME2}" \
        --parameter "github_branch=${GIT_BRANCH}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP2}" \
        --parameter "slice_name=${SLICE_NAME2}"\
        --parameter "ip_prefix=${IP_PREFIX2}"\
        --parameter "client_site=${CLIENT_SITE2}"\
        --parameter "safe_keypair=${SAFE_KEYPAIR2}"\
        --parameter "sdx_server=${SDX_PUBLIC_IP}:8888"\
        --wait \
        ${CLIENT_STACK_NAME2}

In [ ]:
echo SDX client creation complete! 
echo ${CLIENT_NODE_NAME2} status `openstack server show  --format value -c status ${CLIENT_NODE_NAME2}`

CLIENT_PUBLIC_IP2=`openstack server show  --format value -c addresses ${CLIENT_NODE_NAME2} | cut -d " " -f 2`
echo The client public IP is $CLIENT_PUBLIC_IP2

result=255
while [[ "$result" != "0" ]]
do 
    sleep 5
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    exit
	result=$?
done

echo COPY your geni pem to cc@$CLIENT_PUBLIC_IP2:~/geni.pem

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    sudo tail /root/boot.log


#### Client 1 stitch to SDX

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    "cd ~/CICI-SAFE/SDX-Simple &&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'stitch CNode1 192.168.10.2 192.168.10.1/24'"

#### Client 1 advertise its IP prefix

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    "cd ~/CICI-SAFE/SDX-Simple && pwd&&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'route 192.168.10.1/24 192.168.10.2'"

#### Client 2 stitch to SDX

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    "cd ~/CICI-SAFE/SDX-Simple && pwd&&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'stitch CNode1 192.168.20.2 192.168.20.1/24'"

#### Client 2 advertise its IP prefix

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    "cd ~/CICI-SAFE/SDX-Simple && pwd&&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'route 192.168.20.1/24 192.168.20.2'"

#### Connect client 1 and client 2

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    "cd ~/CICI-SAFE/SDX-Simple && pwd&&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'link 192.168.10.1/24 192.168.20.1/24'"

#### Undo stitching

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP1} \
    "cd ~/CICI-SAFE/SDX-Simple &&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'unstitch CNode1'"

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP2} \
    "cd ~/CICI-SAFE/SDX-Simple &&
    sudo ./scripts/sdx_exogeni_client.sh -c ~/client.conf -e 'unstitch CNode1'"

# Please Cleanup Your Resources!!!

#### Delete SDX Stack

In [ ]:
echo Deleting stack ${SDX_STACK_NAME}
openstack stack delete -y --wait ${SDX_STACK_NAME}
echo Stack ${SDX_STACK_NAME} deleted

#### Delete RIAK Stack

In [ ]:
echo Deleting stack ${RIAK_STACK_NAME}
openstack stack delete -y --wait ${RIAK_STACK_NAME}
echo Stack ${RIAK_STACK_NAME} deleted

#### Delete Client Stack

In [ ]:
echo Deleting stack ${CLIENT_STACK_NAME1}
openstack stack delete -y --wait ${CLIENT_STACK_NAME1}
echo Stack ${CLIENT_STACK_NAME1} deleted

In [ ]:
echo Deleting stack ${CLIENT_STACK_NAME2}
openstack stack delete -y --wait ${CLIENT_STACK_NAME2}
echo Stack ${CLIENT_STACK_NAME2} deleted